Part 1: Scraping and Saving HTML Content


1: Navigated to https://sfbay.craigslist.org/search/zip

2:  Interact with the Page-Sorting:
URL when the page is sorted - newest: https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0
URL when the page is sorted - oldest: https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0

Changing the sorting criteria changes the URL - sort=date for newest whereas sort=dateoldest for oldest
Sorting area can be changed by modifying the sort='date' or 'dateoldest in URL in browser's address bar

Changing the sort order is a GET request. It's part of the URL and is not hidden.

And the variable associated is sort.

3: 
URL for page 1 : https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0

URL for page 2 : https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~1~0

URL for page 3 : https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~2~0

Only one number changes when you move pages on the URL which is the number next to 'gallery~'. The number is one less t3:han the page number. For instance, page 1 has 0 next to 'gallery~' in URL and page 2 has 1 there. This number can be changed accordingly to navigate between pages directly from URL

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import re
import os

In [ ]:
time.sleep(10)
# headers = {'User-agent': 'Mozilla/5.0'}

url = 'https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0'
page = requests.get(url) #headers=headers)

print(page)


In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')

# Look at the unmodified source.
print(soup)

There is 'ol' element with class 'cl-static-search-results' which has search results and 'a' element within has link to the item. The code for the same is as follows:

In [ ]:
page_ol = soup.select_one("ol.cl-static-search-results")

In [ ]:
print(page_ol)

In [ ]:
page_ol_a = page_ol.select('a')
print(page_ol_a[0].get('href'))

Alternatively, we can directly search for a however, here you start getting from third index. First two aren't item links.

In [ ]:
page_a = soup.select("a")
print(page_a[3].get('href'))

Let's go back to first method and continue.

In [ ]:
#Check how links we got
print(len(page_ol_a))

Since I got more than 250 links here, I'll pull first 250 using a loop and make a list of the URLs.

In [ ]:
#Store first 250 links in a list
url_list = []
for i in range(0, 250):
    url_list.append(page_ol_a[i].get('href'))

In [ ]:
print(url_list)

5. Save HTML Pages:


In [ ]:
def saveString(html, filename="test.html"):
	try:
		file = open(filename, "w", encoding='utf-8')
		file.write(str(html))
		file.close()
	except Exception as ex:
		print('Error: ' + str(ex))



In [ ]:
for url in url_list:
    time.sleep(5)
    print(url)
    page = requests.get(url) 
    match = re.search(r'/(\d+)\.html$', url)
    id = match.group(1)
    filename = f"{id}.html"
    saveString(page.content,filename)


Part 2: Parsing and Displaying Information from Saved HTML


Read Saved HTML Files:

In [ ]:
# Loop through each file in the directory
directory = os.getcwd()

for filename in os.listdir(directory):

#     # Check if the file ends with .html

    if filename.endswith(".html"):
        # print(filename)

#         # Construct the full file path

        filepath = os.path.join(directory, filename)
        # print(filepath)

#         # Read file to string

        with open(filepath, 'r', encoding='utf-8') as file:

                html = file.read()

#         # ... use html string ...
                soup = BeautifulSoup(html, 'html.parser')
                # print(soup.prettify())
                #Get title
                title_1 = soup.select("span#titletextonly")
                for i in title_1:
                        print(title_1[0].text)
                #Get image source
                img_1 = soup.select("div.gallery")
                img_2 = img_1.select("img")
                print(img_2['src'])
                #Get description
                desc = soup.select("#postingbody")
                for i in desc:
                        print(i.text)
                # #Get post id
                post_id_1 = soup.select("p.postinginfo")
                post_id = post_id_1[1]
                print(post_id_1[1].text)
                #get post date
                #it exists in first time.date timeago
                posted_date = soup.select("time.date.timeago")
                print(posted_date[0].title)
                # #Get update info (If it exists, it exists in second time.date timeago )
                update_info = soup.select('time.date.timeago')
                print(posted_date[1].title)



            



Part 3: Automating Login on The Old Reader


Made an account on https://theoldreader.com/

Using browser’s developer tools to inspect the page and focusing on the <form> tag involved in the login process - `<input>` fields within the login form are utf-8, authenticity_token, user[login], user[password] and commit

 

Looking at the network tab while logging in, it can be seen that POST request was made while logging in. This is to keep login information hidden.
Payload has following items:
utf-8 - ✓
authenticity_token: bmdF4HMov351nBfIZ8R9SbQQUxj6boJ9gGVP10+LEUA=
user[login]
user[password]
commit: Sign In

The input elements are the same as seen on html elements


In [ ]:
headers = {'User-agent': 'Mozilla/5.0'}

url = 'https://theoldreader.com/users/sign_in'
page = requests.get(url, headers=headers)
# print(page)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
input = soup.select_one('#new_user input[name=authenticity_token]')
authenticity_token = input.get('value')
# print(authenticity_token)

time.sleep(5) # 5s


# A session to carry the cookies and make post requests
session = requests.session()

res = session.post('https://theoldreader.com/users/sign_in', 
                        data = {'utf8' : '✓',
                                'authenticity_token' : authenticity_token,
                                'user[login]' : '***', 
                                'user[password]' : '***',
                                'commit' : 'Sign In'}, 
                        timeout = 20)

# Get the Cookies.
cookies = session.cookies.get_dict()
print(cookies)

The session stores session information, user name, a token to remember user and signed at cookies to identify user, and session

In [ ]:
time.sleep(5) # 5s

# To remain in-session.
page_n = requests.get('https://theoldreader.com', cookies=cookies) # or explicitly set cookies of the session (when not using session.xyz)
soup_n = BeautifulSoup(page_n.content, 'html.parser')
print(soup_n.prettify())

In the page, you can see
Bugsnag.user = {
      id: "******",
      name: "******",
      email: "******"
    }

which is my email and name - identified by cookies which means I was able to log in successfully